In [15]:
!pip install catboost


  Using cached catboost-1.2.7-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
Using cached catboost-1.2.7-cp311-cp311-manylinux2014_x86_64.whl (98.7 MB)


In [1]:
from catboost import CatBoostClassifier

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [ ]:
#data_snp_phenotype = pd.read_pickle('/content/data.imputed_geno_groups_subgroups.pkl.gz')

In [ ]:
#data_snp_phenotype = data_snp_phenotype.drop(columns=['Subgroup'])


In [3]:
data_snp_phenotype_filtered = pd.read_csv('/content/data_snp_phenotype_filtered_012.csv.zip', sep=',')

In [4]:
data_snp_phenotype_filtered

,Accession,NC_024459.2,NC_024459.2.1,NC_024459.2.2,NC_024459.2.3,NC_024459.2.4,NC_024459.2.5,NC_024459.2.6,NC_024459.2.7,NC_024459.2.8,...,NC_024459.2.990,NC_024459.2.991,NC_024459.2.992,NC_024459.2.993,NC_024459.2.994,NC_024459.2.995,NC_024459.2.996,NC_024459.2.997,NC_024459.2.998,Heterotic group
0,CRR504157,1|0,1|1,1|0,0|1,1|1,1|0,1|0,1|0,1|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,NSS
1,CRR504163,1|0,1|1,1|0,1|1,1|1,1|0,1|0,1|0,1|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,SPT
2,CRR504172,1|0,1|0,1|0,1|1,1|1,0|1,0|1,0|1,0|1,...,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,NSS
3,CRR504174,0|1,0|1,0|1,0|0,0|0,0|0,0|0,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,1|1,1|1,1|1,PB
4,CRR504181,0|0,1|1,0|0,1|1,1|1,1|0,1|0,1|0,1|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,CRR505744,0|1,1|1,0|1,1|1,1|1,0|1,0|1,0|1,0|1,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,PA
601,CRR505746,0|1,1|1,0|1,1|1,1|1,0|1,0|1,0|1,0|1,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,PA
602,CRR505747,1|0,1|1,1|0,1|1,1|1,1|0,1|0,1|0,1|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,PA
603,CRR505752,1|0,1|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,...,1|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,PA


In [5]:
data_snp_phenotype_filtered.columns

Index(['Accession', 'NC_024459.2', 'NC_024459.2.1', 'NC_024459.2.2',
       'NC_024459.2.3', 'NC_024459.2.4', 'NC_024459.2.5', 'NC_024459.2.6',
       'NC_024459.2.7', 'NC_024459.2.8',
       ...
       'NC_024459.2.990', 'NC_024459.2.991', 'NC_024459.2.992',
       'NC_024459.2.993', 'NC_024459.2.994', 'NC_024459.2.995',
       'NC_024459.2.996', 'NC_024459.2.997', 'NC_024459.2.998',
       'Heterotic group'],
      dtype='object', length=1001)

In [ ]:
#def standardize_genotypes(data_snp_phenotype):
    #for col in data_snp_phenotype.columns[1:]:
        #data_snp_phenotype.loc[:, col] = data_snp_phenotype[col].apply(lambda x: '|'.join(sorted(x.split('|'))))

    #return data_snp_phenotype

#standardize_genotypes(data_snp_phenotype)

#Не хватате ОЗУ, чтобы в один формат привести (чтобы было только, например, С|T, а не С|T и T|C)


In [6]:
#X = data_snp_phenotype.drop(columns=["Heterotic group"]) # Генотипы
#y = data_snp_phenotype["Heterotic group"]  # гетерозисная группа

X = data_snp_phenotype_filtered.drop(columns=["Accession", "Heterotic group"]) # Генотипы
y = data_snp_phenotype_filtered["Heterotic group"]  # гетерозисная группа

In [7]:
def convert_genotype(genotype):
    """Конвертирует строку '1|0', '1|1', '0|1', '0|0', '2|0', '2|1', '0|2', '2|2', '1|2'  в категориальные значения"""
    if genotype == "1|1":
        return 0
    elif genotype == "0|0":
        return 1
    elif genotype == "2|2":
        return 2
    elif genotype == "1|0" or genotype == "0|1":
        return 3
    elif genotype == "2|0" or genotype == "0|2":
        return 4
    else:
        return 5    # "1|2" or "2|1"

In [8]:
X = X.applymap(convert_genotype)

<ipython-input-8-f7d5e4721e2e>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = X.applymap(convert_genotype)


In [9]:
X

,NC_024459.2,NC_024459.2.1,NC_024459.2.2,NC_024459.2.3,NC_024459.2.4,NC_024459.2.5,NC_024459.2.6,NC_024459.2.7,NC_024459.2.8,NC_024459.2.9,...,NC_024459.2.989,NC_024459.2.990,NC_024459.2.991,NC_024459.2.992,NC_024459.2.993,NC_024459.2.994,NC_024459.2.995,NC_024459.2.996,NC_024459.2.997,NC_024459.2.998
0,3,0,3,3,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
1,3,0,3,0,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
2,3,3,3,0,0,3,3,3,3,3,...,0,3,1,1,1,1,1,1,1,1
3,3,3,3,1,1,1,1,1,1,1,...,0,0,1,1,1,1,1,0,0,0
4,1,0,1,0,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,3,0,3,0,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
601,3,0,3,0,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
602,3,0,3,0,0,3,3,3,3,3,...,0,0,1,1,1,1,1,1,1,1
603,3,3,3,1,1,1,1,1,1,1,...,3,0,1,1,1,1,1,1,1,1


In [10]:
y

,Heterotic group
0,NSS
1,SPT
2,NSS
3,PB
4,PA
...,...
600,PA
601,PA
602,PA
603,PA


In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


https://github.com/catboost/catboost/blob/master/catboost/tutorials/custom_loss/custom_loss_and_metric_tutorial.ipynb

https://catboost.ai/docs/en/concepts/loss-functions-multiclassification#objectives-and-metrics

In [12]:
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    cat_features=X.columns.tolist(),
    verbose=100,
    random_seed=42,
    early_stopping_rounds=50,
    used_ram_limit='32gb',
    thread_count=1
)

In [13]:
model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
    verbose=100
)

0:	learn: 0.4607438	test: 0.4500000	best: 0.4500000 (0)	total: 5.33s	remaining: 8m 47s
99:	learn: 0.7520661	test: 0.6500000	best: 0.6833333 (75)	total: 12m 12s	remaining: 0us

bestTest = 0.6833333333
bestIteration = 75

Shrink model to first 76 iterations.


In [14]:
train_accuracy_filtered_data = accuracy_score(y_train, model.predict(X_train))
test_accuracy_filtered_data = accuracy_score(y_test, model.predict(X_test))
print(f"Train accuracy: {train_accuracy_filtered_data}")
print(f"Test accuracy: {test_accuracy_filtered_data}")


Train accuracy: 0.7004132231404959
Test accuracy: 0.6721311475409836


In [18]:
train_precision = precision_score(y_train, model.predict(X_train), average='weighted')
test_precision = precision_score(y_test, model.predict(X_test), average='weighted')
print(f"Train precision: {train_precision}")
print(f"Test precision: {test_precision}")

train_recall = recall_score(y_train, model.predict(X_train), average='weighted')
test_recall = recall_score(y_test, model.predict(X_test), average='weighted')
print(f"Train recall: {train_recall}")
print(f"Test recall: {test_recall}")

train_f1 = f1_score(y_train, model.predict(X_train), average='weighted')
test_f1 = f1_score(y_test, model.predict(X_test), average='weighted')
print(f"Train F1-score: {train_f1}")
print(f"Test F1-score: {test_f1}")

train_roc_auc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
test_roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Test ROC AUC: {test_roc_auc}")



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train precision: 0.6661005496320688
Test precision: 0.6268403727420121
Train recall: 0.7004132231404959
Test recall: 0.6721311475409836
Train F1-score: 0.6717097800234649
Test F1-score: 0.6407861853768044
Train ROC AUC: 0.9328062578550398
Test ROC AUC: 0.8686429164543544
